In [ ]:
# https://github.com/hgupta01/svm_classification_keras/blob/master/SVM%20Layer%20Keras.ipynb
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train.shape)
print(x_test.shape)

num_classes = 10

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples
(60000, 784)
(10000, 784)


In [ ]:
def model_1(x_input):
    x = Dense(512, activation='relu')(x_input)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(256, activation='relu')(x)
    x_out = Dropout(0.5)(x)
    return x_out

In [ ]:
def model_2(x_input):
    x = Dense(800, activation='sigmoid')(x_input)
    x = Dropout(0.5)(x)
    x = Dense(200, activation='sigmoid')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(12)(x)
    return x_out

In [ ]:
inputs = Input(shape=(784,))
x      = model_1(inputs)
x_out  = Dense(10, use_bias=False, activation='linear', name='svm')(x)
model = Model(inputs, x_out)

In [ ]:
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss
    
    return categorical_hinge_loss


In [ ]:
metrics = ['accuracy']
optimizer = tf.keras.optimizers.RMSprop(lr=2e-3, decay=1e-5)
#optimizer = tf.train.AdamOptimizer(1.e-3)

model.compile(optimizer=optimizer, loss=svm_loss(model.get_layer('svm')), metrics=metrics)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
svm (Dense)                  (None, 10)                5120      
Total params: 669,696
Trainable params: 669,696
Non-trainable params: 0
_______________________________________________________

In [ ]:
batch_size = 64
epochs = 4

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/4
938/938 [==============================] - 6s 4ms/step - loss: 10.1532 - accuracy: 0.8488 - val_loss: 10.0542 - val_accuracy: 0.9539
Epoch 2/4
938/938 [==============================] - 3s 3ms/step - loss: 10.0535 - accuracy: 0.9568 - val_loss: 10.0404 - val_accuracy: 0.9683
Epoch 3/4
938/938 [==============================] - 3s 4ms/step - loss: 10.0444 - accuracy: 0.9672 - val_loss: 10.0454 - val_accuracy: 0.9652
Epoch 4/4
938/938 [==============================] - 3s 3ms/step - loss: 10.0378 - accuracy: 0.9722 - val_loss: 10.0392 - val_accuracy: 0.9734
Test loss: 10.03917121887207
Test accuracy: 0.9733999967575073
